In [145]:
from pymongo import MongoClient
import pandas as pd
import re
import folium

ModuleNotFoundError: No module named 'folium'

In [146]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [34]:
db, coll = connectCollection('companies','companies')
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies')

In [35]:
interesting_companies = list(coll.find({"$and":
                                        [{"founded_year":{"$gte":2009}},
                                        {"deadpooled_year":None}]}))

print(len(interesting_companies))


792


In [36]:
interesting_companies = list(coll.find({"$and":
                                        [{"founded_year":{"$gte":2009}},
                                        {"deadpooled_year":None},
                                        {"total_money_raised":{"$exists": True,"$ne":"$0"}},
                                        {"offices.latitude": {"$exists": True, "$ne": None}},
                                        {"offices.longitude": {"$exists": True, "$ne": None}},
                                        {"$or":
                                        [{"category_code":"web"},
                                        {"category_code":"mobile"},
                                        {"category_code":"finance"},
                                        {"category_code":"games_video"},
                                        {"category_code":"analytics"},
                                        {"category_code": "software"}]}]},
                                      {"name":1,"founded_year": 1, "offices.latitude":1,
                                       "offices.longitude":1, "category_code":1, "location":1, 
                                       "total_money_raised":1}))
                                      
print(len(interesting_companies))

42


In [68]:
interesting_companies[0]

{'_id': ObjectId('52cdef7c4bab8bd675297f9e'),
 'name': 'GENWI',
 'category_code': 'mobile',
 'founded_year': 2010,
 'total_money_raised': '$7.1M',
 'offices': [{'latitude': 33.8171, 'longitude': -111.9035}]}

In [83]:
print(len(interesting_companies[0]['offices']))

1


In [130]:
def getLocation(company):
    longitude = company['offices'][0]['longitude']
    latitude = company['offices'][0]['latitude']
    coordinates = [float(longitude), float(latitude)]
    
    return coordinates

getLocation(interesting_companies[0])

coordinates = [{'type': 'Point', 'coordinates':getLocation(comp)}for comp in interesting_companies]

print(coordinates)

[{'type': 'Point', 'coordinates': [-111.9035, 33.8171]}, {'type': 'Point', 'coordinates': [-122.323895, 37.566879]}, {'type': 'Point', 'coordinates': [-122.161523, 37.446823]}, {'type': 'Point', 'coordinates': [-122.3254705, 37.5645375]}, {'type': 'Point', 'coordinates': [-122.214897, 37.772323]}, {'type': 'Point', 'coordinates': [-0.208979, 51.51819]}, {'type': 'Point', 'coordinates': [-71.24762, 42.275263]}, {'type': 'Point', 'coordinates': [-113.515769, 53.544711]}, {'type': 'Point', 'coordinates': [-1.574948, 54.703145]}, {'type': 'Point', 'coordinates': [-122.145153, 37.424614]}, {'type': 'Point', 'coordinates': [-73.956599, 40.768058]}, {'type': 'Point', 'coordinates': [-122.145153, 37.424614]}, {'type': 'Point', 'coordinates': [-73.956599, 40.768058]}, {'type': 'Point', 'coordinates': [-86.0547016, 39.9052263]}, {'type': 'Point', 'coordinates': [-73.987764, 40.744618]}, {'type': 'Point', 'coordinates': [2.1634249, 41.4241228]}, {'type': 'Point', 'coordinates': [-122.0385792, 37.

In [121]:
'''def getLocation(companies):
    location= []
    for i in range(len(companies["offices"])):
        latitude = companies["offices"][i]["latitude"]
        longitude = companies["offices"][i]["longitude"]
        loc = {
            'type':'Point',
            'coordinates':[float(longitude), float(latitude)]
        }    
        location.append(loc)
    return location


getLocation(interesting_companies[0])

for comp in interesting_companies:
    value = {"$set": {"location":getLocation(comp)}}
    coll.update_one(comp, value)'''

[{'type': 'Point', 'coordinates': [-111.9035, 33.8171]}]

In [134]:
companies = pd.DataFrame(interesting_companies)
companies.head()


,_id,name,category_code,founded_year,total_money_raised,offices
0,52cdef7c4bab8bd675297f9e,GENWI,mobile,2010,$7.1M,"[{'latitude': 33.8171, 'longitude': -111.9035}]"
1,52cdef7c4bab8bd675297fec,Fixya,web,2013,$8M,"[{'latitude': 37.566879, 'longitude': -122.323..."
2,52cdef7c4bab8bd67529831a,Social Gaming Network,games_video,2011,$17.1M,"[{'latitude': 37.446823, 'longitude': -122.161..."
3,52cdef7c4bab8bd675298509,Skydeck,mobile,2012,$4M,"[{'latitude': 37.5645375, 'longitude': -122.32..."
4,52cdef7c4bab8bd675298591,Handipoints,games_video,2009,$800k,"[{'latitude': 37.772323, 'longitude': -122.214..."


In [135]:
companies["coordinates"] = coordinates
companies.head()

,_id,name,category_code,founded_year,total_money_raised,offices,coordinates
0,52cdef7c4bab8bd675297f9e,GENWI,mobile,2010,$7.1M,"[{'latitude': 33.8171, 'longitude': -111.9035}]","{'type': 'Point', 'coordinates': [-111.9035, 3..."
1,52cdef7c4bab8bd675297fec,Fixya,web,2013,$8M,"[{'latitude': 37.566879, 'longitude': -122.323...","{'type': 'Point', 'coordinates': [-122.323895,..."
2,52cdef7c4bab8bd67529831a,Social Gaming Network,games_video,2011,$17.1M,"[{'latitude': 37.446823, 'longitude': -122.161...","{'type': 'Point', 'coordinates': [-122.161523,..."
3,52cdef7c4bab8bd675298509,Skydeck,mobile,2012,$4M,"[{'latitude': 37.5645375, 'longitude': -122.32...","{'type': 'Point', 'coordinates': [-122.3254705..."
4,52cdef7c4bab8bd675298591,Handipoints,games_video,2009,$800k,"[{'latitude': 37.772323, 'longitude': -122.214...","{'type': 'Point', 'coordinates': [-122.214897,..."


In [136]:
companies = companies.drop(['_id'], axis = 1)
companies.head()


,name,category_code,founded_year,total_money_raised,offices,coordinates
0,GENWI,mobile,2010,$7.1M,"[{'latitude': 33.8171, 'longitude': -111.9035}]","{'type': 'Point', 'coordinates': [-111.9035, 3..."
1,Fixya,web,2013,$8M,"[{'latitude': 37.566879, 'longitude': -122.323...","{'type': 'Point', 'coordinates': [-122.323895,..."
2,Social Gaming Network,games_video,2011,$17.1M,"[{'latitude': 37.446823, 'longitude': -122.161...","{'type': 'Point', 'coordinates': [-122.161523,..."
3,Skydeck,mobile,2012,$4M,"[{'latitude': 37.5645375, 'longitude': -122.32...","{'type': 'Point', 'coordinates': [-122.3254705..."
4,Handipoints,games_video,2009,$800k,"[{'latitude': 37.772323, 'longitude': -122.214...","{'type': 'Point', 'coordinates': [-122.214897,..."


In [138]:
companies.total_money_raised = companies.total_money_raised.str.replace(r'(\d+)([A-Za-z])', r'\1 \2')
companies.total_money_raised = companies.total_money_raised.str.replace(r'(.)(\d+)', r'\1 \2')
companies.head()


,name,category_code,founded_year,total_money_raised,offices,coordinates
0,GENWI,mobile,2010,$ 7. 1 M,"[{'latitude': 33.8171, 'longitude': -111.9035}]","{'type': 'Point', 'coordinates': [-111.9035, 3..."
1,Fixya,web,2013,$ 8 M,"[{'latitude': 37.566879, 'longitude': -122.323...","{'type': 'Point', 'coordinates': [-122.323895,..."
2,Social Gaming Network,games_video,2011,$ 17. 1 M,"[{'latitude': 37.446823, 'longitude': -122.161...","{'type': 'Point', 'coordinates': [-122.161523,..."
3,Skydeck,mobile,2012,$ 4 M,"[{'latitude': 37.5645375, 'longitude': -122.32...","{'type': 'Point', 'coordinates': [-122.3254705..."
4,Handipoints,games_video,2009,$ 800 k,"[{'latitude': 37.772323, 'longitude': -122.214...","{'type': 'Point', 'coordinates': [-122.214897,..."


In [139]:
#companies3 = companies2[companies2["total_money_raised"].str.contains("M", na=True)]
#companies3
#df[df['model'].str.contains('ac')]

companies1 = companies[['M' in x for x in companies1['total_money_raised']]]
companies1

,name,category_code,founded_year,total_money_raised,offices,coordinates
0,GENWI,mobile,2010,$ 7. 1 M,"[{'latitude': 33.8171, 'longitude': -111.9035}]","{'type': 'Point', 'coordinates': [-111.9035, 3..."
1,Fixya,web,2013,$ 8 M,"[{'latitude': 37.566879, 'longitude': -122.323...","{'type': 'Point', 'coordinates': [-122.323895,..."
2,Social Gaming Network,games_video,2011,$ 17. 1 M,"[{'latitude': 37.446823, 'longitude': -122.161...","{'type': 'Point', 'coordinates': [-122.161523,..."
3,Skydeck,mobile,2012,$ 4 M,"[{'latitude': 37.5645375, 'longitude': -122.32...","{'type': 'Point', 'coordinates': [-122.3254705..."
5,Pixsta,web,2010,£ 3 M,"[{'latitude': 51.51819, 'longitude': -0.208979}]","{'type': 'Point', 'coordinates': [-0.208979, 5..."
6,BrandYourself,web,2009,$ 1. 5 M,"[{'latitude': 42.275263, 'longitude': -71.24762}]","{'type': 'Point', 'coordinates': [-71.24762, 4..."
7,Titan Gaming,games_video,2010,$ 1 M,"[{'latitude': 53.544711, 'longitude': -113.515...","{'type': 'Point', 'coordinates': [-113.515769,..."
13,Formspring,web,2009,$ 14. 3 M,"[{'latitude': 39.9052263, 'longitude': -86.054...","{'type': 'Point', 'coordinates': [-86.0547016,..."
14,Yipit,web,2010,$ 7. 55 M,"[{'latitude': 40.744618, 'longitude': -73.9877...","{'type': 'Point', 'coordinates': [-73.987764, ..."
16,ScaleMP,software,2009,$ 8 M,"[{'latitude': 37.3229727, 'longitude': -122.03...","{'type': 'Point', 'coordinates': [-122.0385792..."


In [140]:
len(companies1)


27

In [148]:
companies1.to_json(orient= "records")

'[{"name":"GENWI","category_code":"mobile","founded_year":2010,"total_money_raised":"$ 7. 1 M","offices":[{"latitude":33.8171,"longitude":-111.9035}],"coordinates":{"type":"Point","coordinates":[-111.9035,33.8171]}},{"name":"Fixya","category_code":"web","founded_year":2013,"total_money_raised":"$ 8 M","offices":[{"latitude":37.566879,"longitude":-122.323895}],"coordinates":{"type":"Point","coordinates":[-122.323895,37.566879]}},{"name":"Social Gaming Network","category_code":"games_video","founded_year":2011,"total_money_raised":"$ 17. 1 M","offices":[{"latitude":37.446823,"longitude":-122.161523},{"latitude":34.0698494,"longitude":-118.4054182}],"coordinates":{"type":"Point","coordinates":[-122.161523,37.446823]}},{"name":"Skydeck","category_code":"mobile","founded_year":2012,"total_money_raised":"$ 4 M","offices":[{"latitude":37.5645375,"longitude":-122.3254705}],"coordinates":{"type":"Point","coordinates":[-122.3254705,37.5645375]}},{"name":"Pixsta","category_code":"web","founded_ye